#### Load data

In [125]:
import numpy as np
import pandas as pd

In [172]:
train = pd.read_csv('data/numerai_training_data.csv')
test = pd.read_csv('data/numerai_tournament_data.csv')

train['is_test'] = 0
test['is_test'] = 1

In [173]:
test['t_id'].shape

(264876,)

In [128]:
data = pd.concat(( train, test ))

data = data.iloc[ np.random.permutation(len( data )) ]
data.reset_index( drop = True, inplace = True )

x = data.drop( ['target', 'is_test', 't_id' ], axis = 1 )
y = data['is_test']

In [129]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.05, random_state=42)

In [130]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=10).fit(x_train, y_train)

In [131]:
x_train = train.drop(['target','is_test'], axis=1)
y_train = train['target']

In [132]:
pred = rf_model.predict(x_train) > 0

In [133]:
select_train = (train[pred > 0]).drop('is_test', axis=1)

In [134]:
select_train.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target
1,0.402661,0.495588,0.550416,0.477632,0.634254,0.464662,0.530294,0.449357,0.561896,0.465977,...,0.352428,0.375022,0.328490,0.511662,0.591985,0.610517,0.519224,0.504858,0.543675,0.0
12,0.492203,0.350288,0.485051,0.651132,0.731687,0.746416,0.488585,0.497216,0.468585,0.778934,...,0.493070,0.604949,0.531637,0.358908,0.349546,0.455286,0.304271,0.460116,0.471477,1.0
14,0.451295,0.753119,0.460834,0.342312,0.579556,0.542030,0.634178,0.387926,0.615693,0.390842,...,0.419197,0.534886,0.330801,0.768373,0.639812,0.862418,0.535595,0.522503,0.567574,1.0
34,0.303139,0.504711,0.595357,0.562585,0.623392,0.582679,0.598352,0.423655,0.528517,0.610704,...,0.389293,0.512826,0.508487,0.467985,0.445168,0.611245,0.434004,0.533969,0.522299,0.0
38,0.437924,0.472173,0.534216,0.416472,0.457939,0.442412,0.405538,0.527336,0.578290,0.458140,...,0.308455,0.583612,0.314289,0.590331,0.586450,0.651290,0.285890,0.379563,0.657725,0.0


In [135]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [136]:
select_train.shape

(14804, 51)

In [137]:
y_class = pd.get_dummies(select_train['target']).values
x_train = select_train.drop('target', axis=1).values

In [194]:
model = Sequential()

model.add(Dense(512, input_dim=(x_train.shape[1]), init='normal'))

for i in range(1,8):
    model.add(Dense(512, init='normal'))
    model.add(Dropout(.4))

model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metric='accuracy')

In [195]:
model.fit(x_train, y_class, nb_epoch=5, batch_size=512, verbose=2, validation_split=0.2)

Train on 11843 samples, validate on 2961 samples
Epoch 1/25
3s - loss: 1.9494 - val_loss: 0.7387
Epoch 2/25
2s - loss: 0.8148 - val_loss: 0.6946
Epoch 3/25
2s - loss: 0.7669 - val_loss: 0.6940
Epoch 4/25
2s - loss: 0.7504 - val_loss: 0.6938
Epoch 5/25
2s - loss: 0.7426 - val_loss: 0.6953
Epoch 6/25
2s - loss: 0.7461 - val_loss: 0.6939
Epoch 7/25
2s - loss: 0.7379 - val_loss: 0.6946
Epoch 8/25
2s - loss: 0.7374 - val_loss: 0.6957
Epoch 9/25
2s - loss: 0.7284 - val_loss: 0.6937
Epoch 10/25
2s - loss: 0.7235 - val_loss: 0.6941
Epoch 11/25
2s - loss: 0.7284 - val_loss: 0.6937
Epoch 12/25
2s - loss: 0.7230 - val_loss: 0.6938
Epoch 13/25
2s - loss: 0.7244 - val_loss: 0.6939
Epoch 14/25
2s - loss: 0.7144 - val_loss: 0.6940
Epoch 15/25
2s - loss: 0.7130 - val_loss: 0.6938
Epoch 16/25
2s - loss: 0.7078 - val_loss: 0.6934
Epoch 17/25
2s - loss: 0.7123 - val_loss: 0.6945
Epoch 18/25
2s - loss: 0.7068 - val_loss: 0.6955
Epoch 19/25
2s - loss: 0.7071 - val_loss: 0.6936
Epoch 20/25
2s - loss: 0.7056

In [187]:
pred = model.predict(test.drop(['t_id', 'is_test'], axis=1).values)

KeyboardInterrupt: 

In [ ]:
print(np.max(pred[:,1]))
print(np.min(pred[:,1]))

In [ ]:
p = pred[:,1]
p.shape

In [ ]:
print(test['t_id'])

In [ ]:
submit = pd.DataFrame({ 't_id':test['t_id'], 'probability':p })
submit.head()

In [ ]:
len(p)

In [ ]:
len(submit.index)

In [ ]:
submit.to_csv('sub_1_sk.csv', index=False)

In [185]:
submit.tail()

,probability,t_id
264871,0.479368,224106
264872,0.507771,50072
264873,0.487209,195265
264874,0.466924,198956
264875,0.443874,208906
